In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import transforms


# Define dataset and transform
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = FaceDataset('train', transform=transform)
test_dataset = FaceDataset('test', transform=transform)

# Define data loader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class DahwinFaceNet(nn.Module):
    def __init__(self):
        super(self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.conv7 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn7 = nn.BatchNorm2d(512)
        self.conv8_1 = nn.Conv2d(512, 128, kernel_size=1, stride=1, padding=0)
        self.conv8_2 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=0)
        self.conv9_1 = nn.Conv2d(128, 512, kernel_size=1, stride=1, padding=0)
        self.conv9_2 = nn.Conv2d(512, 512, kernel_size=7, stride=1, padding=0)
        self.conv10_1 = nn.Conv2d(512, 128, kernel_size=1, stride=1, padding=0)
        self.conv10_2 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=0)
        self.conv11_1 = nn.Conv2d(128, 512, kernel_size=1, stride=1, padding=0)
        self.conv11_2 = nn.Conv2d(512, 512, kernel_size=7, stride=1, padding=0)
        self.fc = nn.Linear(512 * 7 * 7, 128)
        self.bn8 = nn.BatchNorm1d(128)

    def forward(self, x):
      # Feature extraction
      x = F.relu(self.bn1(self.conv1(x)))
      x = F.max_pool2d(F.relu(self.bn2(self.conv2(x))), 2)
      x = F.relu(self.bn3(self.conv3(x)))
      x = F.max_pool2d(F.relu(self.bn4(self.conv4(x))), 2)
      x = F.relu(self.bn5(self.conv5(x)))
      x = F.max_pool2d(F.relu(self.bn6(self.conv6(x))), 2)
      x = F.relu(self.bn7(self.conv7(x)))
      x = F.max_pool2d(x, 2)
      x = F.relu(self.conv8_1(self.conv8_2(x)))
      x = F.relu(self.conv9_1(self.conv9_2(x)))
      x = F.relu(self.conv10_1(self.conv10_2(x)))
      x = F.relu(self.conv11_1(self.conv11_2(x)))
      # Flatten
      x = x.view(-1, 512 * 7 * 7)
      # Fully connected layer and L2 normalization
      x = self.fc(x)
      x = F.normalize(self.bn8(x), p=2, dim=1)
      return x
torch.manual_seed(42)
model = DahwinFaceNet()




criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model
for epoch in range(10):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print('Epoch: {}, Test Accuracy: {}%'.format(epoch, accuracy))

# Save model
torch.save(model.state_dict(), 'model.pth')


In [ ]:
    # Evaluate model
    model.eval()
    correct = 0
    total = 0
    test_loss = 0  # initialize test loss
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * labels.size(0)  # accumulate test loss
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        test_loss /= total  # calculate average test loss
        print('Epoch: {}, Test Accuracy: {}%, Test Loss: {:.4f}'.format(epoch, accuracy, test_loss))